In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
import numpy as np

In [2]:
BATCH_SIZE = 128
EPOCHS = 10
LAYER_SIZE = 10
S_LAYER_SIZE = 7
LATENT_SIZE = 4

In [3]:
# Load the data from the file

saved = np.load("testing1.npy", allow_pickle=True)

# Make a trainging and testing batch
train_data = torch.Tensor(saved[:int(len(saved)*0.5)])
test_data = torch.Tensor(saved[int(len(saved)*0.5):])
print(train_data.shape)
INPUT_SIZE = len(test_data[0])
INPUT_SIZE

torch.Size([50000, 11])


11

In [4]:
device = "cuda"

In [5]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        # Encoder
        self.l1 = nn.Linear(INPUT_SIZE, LAYER_SIZE)
        self.ltest1 = nn.Linear(LAYER_SIZE, S_LAYER_SIZE)
        self.l2a = nn.Linear(S_LAYER_SIZE, LATENT_SIZE)
        self.l2b = nn.Linear(S_LAYER_SIZE, LATENT_SIZE)
        
        # Decoder
        self.l3 = nn.Linear(LATENT_SIZE, S_LAYER_SIZE)
        self.ltest2 = nn.Linear(S_LAYER_SIZE, LAYER_SIZE)
        self.l4 = nn.Linear(LAYER_SIZE, INPUT_SIZE)
            
    # Run some data through the encoder
    def encode(self, x):
        out = F.relu(self.l1(x))
        out = F.relu(self.ltest1(out))
        # return the mu and the sigma
        return self.l2a(out), self.l2b(out)
    
    # The reparameterization trick, taken from https://github.com/pytorch/examples/blob/master/vae/main.py
    def reparameterize(self, mu, sigma):
        std = torch.exp(0.5*sigma)
        eps = torch.randn_like(std)
        return mu + eps*std
    
    def decode(self, x):
        out = F.relu(self.l3(x))
        out = F.relu(self.ltest2(out))
        return torch.sigmoid(self.l4(out)) # sigmoid vs tanh
    
    def forward(self, x):
        # TODO: check the shape of x to be sure we have the right input
        mu, sigma = self.encode(x)
        z = self.reparameterize(mu, sigma)
        # The loss function needs the mu and the sigma so just return them here
        return self.decode(z), mu, sigma

In [14]:
# Reconstruction + KL divergence losses summed over all elements and batch
# Taken from https://github.com/pytorch/examples/blob/master/vae/main.py
def loss_function(recon_x, x, mu, sigma):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + sigma - mu.pow(2) - sigma.exp())
    return BCE + KLD

In [7]:
def train(e_count, model, optimizer):
    data = train_data
    model.train()
    train_loss = 0
    for i in range(0, len(data), BATCH_SIZE):
        batch = data[i:i+BATCH_SIZE].to(device)
        model.zero_grad()
        recons, mu, sigma = model(batch)
        loss = loss_function(recons, batch, mu, sigma)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
    print(f" -- Epoch: {e_count}, Loss: {train_loss/len(data)}")        
        
    
    

In [8]:
# Lets use CPU
def test(e_count, model, optimizer):
    model.eval()
    test_loss = 0
    for i in range(0, len(test_data), BATCH_SIZE):
        batch = test_data[i:i+BATCH_SIZE].to(device)
        recons, mu, sigma = model(batch)
        loss = loss_function(recons, batch, mu, sigma)
        test_loss += loss.item()
    print(f"TEST Epoch: {e_count}, Loss: {test_loss/len(test_data)}") 
        
        

In [9]:
def get_data(data, balanced, size):
    res = []
    for d in data:
        if d[10] == int(balanced):
            res.append(d)
        if len(res) == size:
            break
    return torch.Tensor(res)

In [10]:
train_data=train_data[torch.randperm(train_data.size()[0])]

In [11]:
# TWO DATA SETS, TWO MODELS
vae1 = VAE().to(device)
opt1 = optim.Adam(vae1.parameters(), lr=0.001)

In [12]:
for i in range(EPOCHS):
    train(i, vae1, opt1)

 -- Epoch: 0, Loss: 0.0
 -- Epoch: 1, Loss: 0.0
 -- Epoch: 2, Loss: 0.0
 -- Epoch: 3, Loss: 0.0
 -- Epoch: 4, Loss: 0.0
 -- Epoch: 5, Loss: 0.0
 -- Epoch: 6, Loss: 0.0
 -- Epoch: 7, Loss: 0.0
 -- Epoch: 8, Loss: 0.0
 -- Epoch: 9, Loss: 0.0


In [15]:
for i in range(EPOCHS):
    test(i, vae1, opt1)

TEST Epoch: 0, Loss: 6.903801094970703
TEST Epoch: 1, Loss: 6.903724771728515
TEST Epoch: 2, Loss: 6.903820484619141
TEST Epoch: 3, Loss: 6.903682515869141
TEST Epoch: 4, Loss: 6.903768715820313
TEST Epoch: 5, Loss: 6.903703172607422
TEST Epoch: 6, Loss: 6.903784884033203
TEST Epoch: 7, Loss: 6.9038821557617185
TEST Epoch: 8, Loss: 6.90377470703125
TEST Epoch: 9, Loss: 6.903785372314453


In [ ]:
from torch.autograd import Variable

res = [0, 0, 0]
k = []
for i in range(10000):
    sample = Variable(torch.randn(1, LATENT_SIZE))
    a = vae1.decode(sample.to("cuda"))[0]
    num = float(sum(list(a[10:])))
    k.append(num)
    if num < 0.2:
        res[0] += 1
    elif num <0.5:
        res[1] += 1
    else:
        res[2] += 1
        
d = 0
nd = 0

for a in test_data:
    if a[10] == 1:
        d += 1
    else:
        nd += 1
        
        
print(f"UNBALANCED DATASET")
print(f"counts of intervals: [0, 0.1]: {res[0]}, [0.1, 0.7]: {res[1]}, [0.7, 1]: {res[2]}")
print(f"-- so the ratio to done and not done (deleting the middle ones) is {res[2]/(res[2]+res[0])}")
print(f"The real ration is {d/(nd+d)}")

In [ ]:
sample = Variable(torch.randn(2, LATENT_SIZE))
res = vae1.decode(sample.to("cuda"))

In [ ]:
res